In [ ]:
import pandas as pd

#df = pd.read_csv('/content/drive/MyDrive/datasets/sample_transformers/train.csv')
#df = pd.read_csv('Joel\\Transformers\\TrialbotDatabaseProcessingTools\\results06_09_2022-124336.csv', encoding='utf-8',  on_bad_lines='skip', sep=";")
df = pd.read_csv('Joel\\datasets\\df_train_averaged_classification.csv', encoding='utf-8',  on_bad_lines='skip')
df.head()

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Arbeitskollegin. Auf einer Hochzeit. Seife gek...,0,1,1,1,1,"(0, 1, 1, 1, 1)"
1,Auf Empfehlung. Als ein Freund eine Story erzä...,0,1,1,1,1,"(0, 1, 1, 1, 1)"
2,Bei einer Teststation beim Josef. Als ich mit ...,1,1,1,1,1,"(1, 1, 1, 1, 1)"
3,Berufsschule. In der famiele. Heute morgen auf...,0,0,1,1,1,"(0, 0, 1, 1, 1)"
4,"Besuch im Josephs. Keine konkrete Erinnerung, ...",0,1,0,1,1,"(0, 1, 0, 1, 1)"


In [ ]:
df.shape[0]

80

In [ ]:
df.describe()

,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin
count,80.000000,80.000000,80.000000,80.000000,80.000000
mean,0.487500,0.537500,0.487500,0.600000,0.512500
std,0.502997,0.501737,0.502997,0.492989,0.502997
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
print(df.isnull())
df = df.dropna(how='all')

     text  neuro_bin  extra_bin  off_bin  ver_bin  gew_bin  labels
0   False      False      False    False    False    False   False
1   False      False      False    False    False    False   False
2   False      False      False    False    False    False   False
3   False      False      False    False    False    False   False
4   False      False      False    False    False    False   False
..    ...        ...        ...      ...      ...      ...     ...
75  False      False      False    False    False    False   False
76  False      False      False    False    False    False   False
77  False      False      False    False    False    False   False
78  False      False      False    False    False    False   False
79  False      False      False    False    False    False   False

[80 rows x 7 columns]


In [ ]:
print(df.isnull())

     text  neuro_bin  extra_bin  off_bin  ver_bin  gew_bin  labels
0   False      False      False    False    False    False   False
1   False      False      False    False    False    False   False
2   False      False      False    False    False    False   False
3   False      False      False    False    False    False   False
4   False      False      False    False    False    False   False
..    ...        ...        ...      ...      ...      ...     ...
75  False      False      False    False    False    False   False
76  False      False      False    False    False    False   False
77  False      False      False    False    False    False   False
78  False      False      False    False    False    False   False
79  False      False      False    False    False    False   False

[80 rows x 7 columns]


In [ ]:
# Inverse Meaning Items have already been recoded before saving into DB
neuro_values = ["neu_item_6","neu_item_21","neu_item_11","neu_item_26","neu_item_41","neu_item_51"]
extra_values = ["ext_item_2","ext_item_7","ext_item_22","ext_item_32","ext_item_37","next_item_52"] # Typo when saving ext_item_52 to db.. but does not matter
off_values = ["off_item_8","off_item_13","off_item_23","off_item_43","off_item_48","off_item_58"]
ver_values = ["ver_item_9", "ver_item_14", "ver_item_24", "ver_item_39","ver_item_49","ver_item_59"]
gew_values = ["gew_item_5","gew_item_10","gew_item_20","gew_item_40","gew_item_50","gew_item_55"]

def Average(l):
    l = list(map(int, l))
    return sum(l) / len(l)


# Scale factor according to the NEO-FFI manual.
def Scale(i):
    i = i * 12
    return i

def compute(dimension, row):
    values = []
    if dimension == "neuro":
        values = [row[i] for i in neuro_values]
    if dimension == "extra":
        values = [row[i] for i in extra_values]
    if dimension == "off":
        values = [row[i] for i in off_values]
    if dimension == "ver":
        values = [row[i] for i in ver_values]    
    if dimension == "gew":
        values = [row[i] for i in gew_values]    
    mean = Average(values)
    # Scale mean values according to NEO-FFI manual
    # and convert into percentage, with
    # 60 being the max value
    max_value = 60
    skala = (Scale(mean) * 100) / max_value

    return round(skala)

In [ ]:
df['neuro_value'] = df.apply(lambda row: compute("neuro",row), axis=1)
df['extra_value'] = df.apply(lambda row: compute("extra",row), axis=1)
df['off_value'] = df.apply(lambda row: compute("off",row), axis=1)
df['ver_value'] = df.apply(lambda row: compute("ver",row), axis=1)
df['gew_value'] = df.apply(lambda row: compute("gew",row), axis=1)
df.head

<bound method NDFrame.head of                                  how_became_aware     gender  \
0                                 Arbeitskollegin   Männlich   
1                                  Auf Empfehlung   Männlich   
2                Bei einer Teststation beim Josef   Weiblich   
3                                    Berufsschule   Männlich   
4                               Besuch im Josephs   Weiblich   
..                                            ...        ...   
78               Über das Jacobs Innovationslabor   Weiblich   
79                           Über einen Bekannten   Weiblich   
80  Über einen geteilten Beitrag auf Social Media   Männlich   
81                              über das Institut   Weiblich   
82                               über das Josephs  kA-Gender   

   sexual_orientation  neu_item_6  neu_item_21  neu_item_11  neu_item_26  \
0       Heterosexuell         4.0          2.0          2.0          2.0   
1       Heterosexuell         2.0          3.0   

In [ ]:
## Binarize Big Five dimension scores:
# Scores > 50 = 1 (true)
# Scores <= 50 = 0 (false)
def binarize_scores(score):
  if score > 50:
    return 1
  else:
    return 0

In [ ]:
#df['text'] = df[['how_became_aware', 'ext_item_7_text', 'gew_item_20_text', 'ext_item_32_text', 'off_item_48_text', 'gew_item_55_text', 'soc_essay']].apply(".".join, axis=1) # chatbot_experience missing for now
df_training = df.copy()

df_training['neuro_bin'] = df_training.apply(lambda row: binarize_scores(row['neuro_value']), axis=1)
df_training['extra_bin'] = df_training.apply(lambda row: binarize_scores(row['extra_value']), axis=1)
df_training['off_bin'] = df_training.apply(lambda row: binarize_scores(row['off_value']), axis=1)
df_training['ver_bin'] = df_training.apply(lambda row: binarize_scores(row['ver_value']), axis=1)
df_training['gew_bin'] = df_training.apply(lambda row: binarize_scores(row['gew_value']), axis=1)

KeyError: ignored

In [ ]:
df_training.head()

,how_became_aware,gender,sexual_orientation,neu_item_6,neu_item_21,neu_item_11,neu_item_26,neu_item_41,neu_item_51,ext_item_2,...,extra_value,off_value,ver_value,gew_value,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin
0,Arbeitskollegin,Männlich,Heterosexuell,4.0,2.0,2.0,2.0,3.0,3.0,5.0,...,83,97,97,80,Arbeitskollegin.Auf einer Hochzeit.Seife gekau...,1,1,1,1,1
1,Auf Empfehlung,Männlich,Heterosexuell,2.0,3.0,3.0,2.0,2.0,2.0,4.0,...,73,73,87,90,Auf Empfehlung.Als ein Freund eine Story erzäh...,0,1,1,1,1
2,Bei einer Teststation beim Josef,Weiblich,kA-SexOrientation,3.0,3.0,4.0,3.0,3.0,4.0,3.0,...,67,70,80,87,Bei einer Teststation beim Josef.Als ich mit m...,1,1,1,1,1
3,Berufsschule,Männlich,Heterosexuell,1.0,2.0,2.0,1.0,1.0,1.0,3.0,...,63,77,90,97,Berufsschule.In der famiele.Heute morgen aufzu...,0,1,1,1,1
4,Besuch im Josephs,Weiblich,Heterosexuell,2.0,2.0,2.0,1.0,2.0,2.0,4.0,...,80,67,93,90,"Besuch im Josephs.Keine konkrete Erinnerung, a...",0,1,1,1,1


In [ ]:
df_training['text'][:4]

0    Arbeitskollegin. Auf einer Hochzeit. Seife gek...
1    Auf Empfehlung. Als ein Freund eine Story erzä...
2    Bei einer Teststation beim Josef. Als ich mit ...
3    Berufsschule. In der famiele. Heute morgen auf...
Name: text, dtype: object

In [ ]:
df_training = df_training[['text', 'neuro_bin', 'extra_bin', 'off_bin', 'ver_bin', 'gew_bin']]
df_training.head()

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin
0,Arbeitskollegin.Auf einer Hochzeit.Seife gekau...,1,1,1,1,1
1,Auf Empfehlung.Als ein Freund eine Story erzäh...,0,1,1,1,1
2,Bei einer Teststation beim Josef.Als ich mit m...,1,1,1,1,1
3,Berufsschule.In der famiele.Heute morgen aufzu...,0,1,1,1,1
4,"Besuch im Josephs.Keine konkrete Erinnerung, a...",0,1,1,1,1


In [ ]:
# text field is required for simpletransformers
df_training['text'] = df_training['text'].apply(lambda x: x.replace('\n', ' '))
df_training.head()

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Arbeitskollegin. Auf einer Hochzeit. Seife gek...,0,1,1,1,1,"(0, 1, 1, 1, 1)"
1,Auf Empfehlung. Als ein Freund eine Story erzä...,0,1,1,1,1,"(0, 1, 1, 1, 1)"
2,Bei einer Teststation beim Josef. Als ich mit ...,1,1,1,1,1,"(1, 1, 1, 1, 1)"
3,Berufsschule. In der famiele. Heute morgen auf...,0,0,1,1,1,"(0, 0, 1, 1, 1)"
4,"Besuch im Josephs. Keine konkrete Erinnerung, ...",0,1,0,1,1,"(0, 1, 0, 1, 1)"


In [ ]:
# simpletransformers also expects a multi-hot encoded list of labels
df_training = df.copy()
df_training['labels'] = list(zip(df_training.neuro_bin.tolist(), df_training.extra_bin.tolist(), df_training.off_bin.tolist(), df_training.ver_bin.tolist(),  df_training.gew_bin.tolist()))


In [ ]:
# Find out max string length of text column
print(int(df_training["text"].astype(str).str.len().max()))

1353


In [ ]:
#df_train_padding = df_training.copy()
#df_train_padding['text']=df_train_padding.text.str.pad(2048,side='right',fillchar=' ')

In [ ]:
#df_train_padding.iloc[10]["text"]

'Durch Freunde.Hiding the Pain Harold angeschaut.Ich habe einen riesigen Tisch für eine Rücksendung in einen eigentlich ungeeigneten Karton verpackt..Ich fühle mich im Reinen.Natürliche Medizin. Es gibt so gutes altes Wissen darüber und wir müssen uns von der Pharmaindustrieabhängigkeit befreien.Ich fange einfach damit an.An einem sonnigen Tag mit dem Cabrio ins Grüne fahren. Einfach wohin die Straße führt, ohne Ziel.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
#df_train_padding['text'].str.len().mean()

2048.0

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
df_training.shape[0]

80

In [ ]:
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df_training, test_size=0.2)

In [ ]:
train_df.head()

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
65,durch die TH. ist mir zu persönlich. ist mir z...,1,1,1,0,0,"(1, 1, 1, 0, 0)"
20,Durch einen Rundgang. Weiß ich nicht mehr. Diä...,0,1,0,0,1,"(0, 1, 0, 0, 1)"
28,Habe dich im JOSEPHS entdeckt!. Bei roten Pand...,0,1,1,1,1,"(0, 1, 1, 1, 1)"
79,über das Josephs. beim Weinfest mit Freundinne...,0,1,1,1,1,"(0, 1, 1, 1, 1)"
56,Testuser. Film. Arbeit erledigt. Passiert selt...,1,0,1,0,1,"(1, 0, 1, 0, 1)"


In [ ]:
train_df.shape[0]

64

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, use_cuda=True,
                                      args={'train_batch_size':4, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'c

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
# Train the model
model.train_model(train_df)

  0%|          | 0/64 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/16 [00:00<?, ?it/s]

(3, 0.6992468597988287)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(result)

{'LRAP': 0.8601307189542481, 'eval_loss': 0.6822028557459513}


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 43 to 29
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       66 non-null     object
 1   neuro_bin  66 non-null     int64 
 2   extra_bin  66 non-null     int64 
 3   off_bin    66 non-null     int64 
 4   ver_bin    66 non-null     int64 
 5   gew_bin    66 non-null     int64 
 6   labels     66 non-null     object
dtypes: int64(5), object(2)
memory usage: 4.1+ KB


In [ ]:
preds, outputs = model.predict(["Durch meine Schwester.Heute im Waldkindergarten.Bin heute mit dem Stoff der letzten Klausur fertig geworden.Sehr gut.Weiter.Weiter.Picknicken an einer ruhigen Stelle mit guter aussicht und schönen Musik während man den Sonnenuntergang betrachtet"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sub_df = pd.DataFrame(outputs, columns=['neuro_bin', 'extra_bin', 'off_bin', 'ver_bin', 'gew_bin'])
sub_df.head()

,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin
0,0.507812,0.480713,0.496094,0.52832,0.543945


In [ ]:
print(preds)

[[1, 0, 0, 1, 1]]


In [ ]:
print(eval_df.iloc[1])

text         Joseph.Wie sich jemand vom E-Roller hingelegt ...
neuro_bin                                                    1
extra_bin                                                    1
off_bin                                                      1
ver_bin                                                      1
gew_bin                                                      1
labels                                         (1, 1, 1, 1, 1)
Name: 39, dtype: object


In [ ]:
print(eval_df.iloc[1].text)

Durch den Beta-Test.Mit Freunden auf einer Geburtstagsfeier..Ich habe wichtige Mails geschrieben..Toll, dann kann ich viel kreatives schaffen..Biologie und das menschliche Verhalten..Ich mache Pläne und bespreche mich mit Menschen....Ein ständiges Lächeln, ein Herz voller Freude, Schmetterlinge im Bauch, und der Wunsch, dass dieses Date nie endet sind sichere Zeichen, dass es das perfekte Date ist.  Aber wie kommt man dazu?  Viele Menschen, viele Meinungen. Jeder hat seine Vorstellung, wie das perfekte Date aussehen soll. Einige mögen einen ruhigen Abend, andere eine Kneipentour.  Für die einen ist es ins Kino zu gehen, für die anderen ins Restaurant, und für wieder andere in der Natur.  Eine Sache, die aber sicher ist, obwohl wo und was man macht eine große Rolle spielt, ist mit wem man es macht noch wichtiger. Nur mit einer besonderen Person kann es wirklich perfekt sein.  Wenn du deine besondere Person schon hast, bist du schon auf dem halben Weg zum perfekten Date.  Wenn du aber je

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel


model2 = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5,
                                      args={'train_batch_size':2, 'gradient_accumulation_steps':16, 
                                            'learning_rate': 3e-5, 'num_train_epochs': 3})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias',

In [ ]:
# Train the model
model2.train_model(train_df)

  0%|          | 0/66 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/33 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/33 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/33 [00:00<?, ?it/s]

(6, 0.6734947332491478)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)

  0%|          | 0/17 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.8418300653594769, 'eval_loss': 0.7085650364557902}


In [ ]:

def binarize_scores_average(dimension, value):
    if dimension == "neuro":
        if value > 56:
          return 1
    if dimension == "extra":
        if value > 65:
          return 1
    if dimension == "off":
        if value > 69:
          return 1
    if dimension == "ver":
        if value > 69:
          return 1  
    if dimension == "gew":
        if value > 76:
          return 1 

    return 0

In [ ]:
df_train_averaged_classification = df.copy()

In [ ]:
df_train_averaged_classification['neuro_bin'] = df.apply(lambda row: binarize_scores_average("neuro", row['neuro_value']), axis=1)
df_train_averaged_classification['extra_bin'] = df.apply(lambda row: binarize_scores_average("extra", row['extra_value']), axis=1)
df_train_averaged_classification['off_bin'] = df.apply(lambda row: binarize_scores_average("off", row['off_value']), axis=1)
df_train_averaged_classification['ver_bin'] = df.apply(lambda row: binarize_scores_average("ver", row['ver_value']), axis=1)
df_train_averaged_classification['gew_bin'] = df.apply(lambda row: binarize_scores_average("gew", row['gew_value']), axis=1)
df_train_averaged_classification.head()

,how_became_aware,gender,sexual_orientation,neu_item_6,neu_item_21,neu_item_11,neu_item_26,neu_item_41,neu_item_51,ext_item_2,...,extra_value,off_value,ver_value,gew_value,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin
0,Arbeitskollegin,Männlich,Heterosexuell,4.0,2.0,2.0,2.0,3.0,3.0,5.0,...,83,97,97,80,Arbeitskollegin.Auf einer Hochzeit.Seife gekau...,0,1,1,1,1
1,Auf Empfehlung,Männlich,Heterosexuell,2.0,3.0,3.0,2.0,2.0,2.0,4.0,...,73,73,87,90,Auf Empfehlung.Als ein Freund eine Story erzäh...,0,1,1,1,1
2,Bei einer Teststation beim Josef,Weiblich,kA-SexOrientation,3.0,3.0,4.0,3.0,3.0,4.0,3.0,...,67,70,80,87,Bei einer Teststation beim Josef.Als ich mit m...,1,1,1,1,1
3,Berufsschule,Männlich,Heterosexuell,1.0,2.0,2.0,1.0,1.0,1.0,3.0,...,63,77,90,97,Berufsschule.In der famiele.Heute morgen aufzu...,0,0,1,1,1
4,Besuch im Josephs,Weiblich,Heterosexuell,2.0,2.0,2.0,1.0,2.0,2.0,4.0,...,80,67,93,90,"Besuch im Josephs.Keine konkrete Erinnerung, a...",0,1,0,1,1


In [ ]:
df_train_averaged_classification = df_train_averaged_classification[['text', 'neuro_bin', 'extra_bin', 'off_bin', 'ver_bin', 'gew_bin']]
df_train_averaged_classification['text'] = df_train_averaged_classification['text'].apply(lambda x: x.replace('\n', ' '))
df_train_averaged_classification['labels'] = list(zip(df_train_averaged_classification.neuro_bin.tolist(), df_train_averaged_classification.extra_bin.tolist(), df_train_averaged_classification.off_bin.tolist(), df_train_averaged_classification.ver_bin.tolist(),  df_train_averaged_classification.gew_bin.tolist()))
df_train_averaged_classification.head()

train_df_averaged_classification, eval_df_averaged_classification = train_test_split(df_train_averaged_classification, test_size=0.2)


C:\Users\Max\AppData\Local\Temp\ipykernel_22080\1429786955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_averaged_classification['text'] = df_train_averaged_classification['text'].apply(lambda x: x.replace('\n', ' '))
C:\Users\Max\AppData\Local\Temp\ipykernel_22080\1429786955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_averaged_classification['labels'] = list(zip(df_train_averaged_classification.neuro_bin.tolist(), df_train_averaged_classification.extra_bin.tolist(), df_train

In [ ]:
train_df_averaged_classification.head()

,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
9,Durch Empfehlung.Über das Hide the Pain Harold...,0,0,0,0,0,"(0, 0, 0, 0, 0)"
21,Durch einen Schulausflug zur FAU in Nürnberg.E...,1,0,0,0,0,"(1, 0, 0, 0, 0)"
37,Institut.Sonntag als meine Schwester einen Wit...,0,0,1,1,1,"(0, 0, 1, 1, 1)"
30,Ich bin hier einfach reingelaufen.An einem Abe...,0,1,1,1,0,"(0, 1, 1, 1, 0)"
5,Das Zukunftsmuseum (?) in nürnberg.Als ich ein...,1,0,1,0,0,"(1, 0, 1, 0, 0)"


In [ ]:
model.train_model(train_df_averaged_classification)

  0%|          | 0/66 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/17 [00:00<?, ?it/s]

(3, 0.7358173181613287)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df_averaged_classification)
print(result)

  0%|          | 0/17 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.7673202614379084, 'eval_loss': 0.6847199400266012}


In [ ]:
result2, model_outputs2, wrong_predictions2 = model.eval_model(eval_df)
print(result2)

  0%|          | 0/17 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'LRAP': 0.7988562091503266, 'eval_loss': 0.6789486209551493}


# Binary Classification

### Neuroticism

#### Neuroticism: Robert Base Simple

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Section
from sklearn.metrics import f1_score
import pandas as pd
import torch
print(torch.cuda.is_available())
train_df_neuro = pd.read_csv('Joel\\datasets\\sds\\transf\\train.csv')
eval_df_neuro = pd.read_csv('Joel\\datasets\\sds\\transf\\test.csv')
train_df_neuro['labels'] = train_df_neuro.neuro_bin
eval_df_neuro['labels'] = eval_df_neuro.neuro_bin
train_df_neuro.head()

True


,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch eine Freundin von mir. Ich habe sehr gel...,1,1,1,1,0,1
1,Durch einen Schulausflug zur FAU in Nürnberg. ...,1,0,0,0,0,1
2,Ich bin hier einfach reingelaufen. An einem Ab...,0,1,1,1,0,0
3,"Email von einer Kollegin. Gestern, gemeinsam m...",0,1,1,0,0,0
4,Über Arbeitskollegen. Mir hat letztens jemand ...,1,0,0,1,1,1


In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)

# Create a ClassificationModel
model_bin = ClassificationModel(
    "roberta", "roberta-base", args=model_args_bin, use_cuda = True
)
# Train the model
model_bin.train_model(train_df_neuro)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.5930185590471541)

In [ ]:
result_bin, model_outputs_bin, wrong_predictions_bin = model_bin.eval_model(eval_df_neuro, f1=f1_score)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(result_bin)

{'mcc': 0.5163977794943222, 'tp': 7, 'tn': 5, 'fp': 3, 'fn': 1, 'auroc': 0.90625, 'auprc': 0.928409090909091, 'f1': 0.7777777777777777, 'eval_loss': 0.498992919921875}


#### Neuroticism: bert-german-cased

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)

# Create a ClassificationModel
model_bin = ClassificationModel(
    'bert', 'bert-base-german-cased', args=model_args_bin, use_cuda = True
)
# Train the model
model_bin.train_model(train_df_neuro)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.1951525526387351)

In [ ]:
result_bin, model_outputs_bin, wrong_predictions_bin = model_bin.eval_model(eval_df_neuro, f1=f1_score)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(result_bin)

{'mcc': 0.5163977794943222, 'tp': 5, 'tn': 7, 'fp': 1, 'fn': 3, 'auroc': 0.90625, 'auprc': 0.9115936147186148, 'f1': 0.7142857142857143, 'eval_loss': 1.0135351419448853}


#### Neuroticism: distilbert

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)

# Create a ClassificationModel
model_bin = ClassificationModel(
    'distilbert', 'distilbert-base-multilingual-cased', args=model_args_bin, use_cuda = True
)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'pre_classif

In [ ]:
result_bin, model_outputs_bin, wrong_predictions_bin = model_bin.eval_model(eval_df_neuro, f1=f1_score)
print(result_bin)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.48038446141526137, 'tp': 8, 'tn': 3, 'fp': 5, 'fn': 0, 'auroc': 0.625, 'auprc': 0.6838945776445776, 'f1': 0.761904761904762, 'eval_loss': 0.68780517578125}


### Extraversion

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Section
from sklearn.metrics import f1_score
import pandas as pd
import torch
print(torch.cuda.is_available())
train_df = pd.read_csv('Joel\\datasets\\sds\\transf\\train.csv')
eval_df = pd.read_csv('Joel\\datasets\\sds\\transf\\test.csv')
train_df['labels'] = train_df.extra_bin
eval_df['labels'] = eval_df.extra_bin
train_df.head()

True


,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch eine Freundin von mir. Ich habe sehr gel...,1,1,1,1,0,1
1,Durch einen Schulausflug zur FAU in Nürnberg. ...,1,0,0,0,0,0
2,Ich bin hier einfach reingelaufen. An einem Ab...,0,1,1,1,0,1
3,"Email von einer Kollegin. Gestern, gemeinsam m...",0,1,1,0,0,1
4,Über Arbeitskollegen. Mir hat letztens jemand ...,1,0,0,1,1,0


#### Extraversion roberta-base
result f1 was: 0.68

#### Extraversion: bert german cased

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'bert', 'bert-base-german-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.2304175853729248)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.6831300510639732, 'tp': 7, 'tn': 6, 'fp': 3, 'fn': 0, 'auroc': 0.8571428571428572, 'auprc': 0.8559523809523809, 'f1': 0.8235294117647058, 'eval_loss': 1.0235826969146729}


#### Extraversion distilbert

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'distilbert', 'distilbert-base-multilingual-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.4393265587942941)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.6831300510639732, 'tp': 7, 'tn': 6, 'fp': 3, 'fn': 0, 'auroc': 0.9365079365079365, 'auprc': 0.9325396825396826, 'f1': 0.8235294117647058, 'eval_loss': 0.5073041915893555}


In [ ]:
predictions, raw_outputs = model_bin.predict(list(eval_df['text']))

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print("Predictions: " + str(predictions))
print("Real: " + str(list(eval_df['labels'])))

Predictions: [1 0 1 1 1 1 0 0 1 0 0 1 1 1 1 0]
Real: [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]


## Openness

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Section
from sklearn.metrics import f1_score
import pandas as pd
import torch
print(torch.cuda.is_available())
train_df = pd.read_csv('Joel\\datasets\\sds\\transf\\train.csv')
eval_df = pd.read_csv('Joel\\datasets\\sds\\transf\\test.csv')
train_df['labels'] = train_df.off_bin
eval_df['labels'] = eval_df.off_bin
train_df.head()

True


,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch eine Freundin von mir. Ich habe sehr gel...,1,1,1,1,0,1
1,Durch einen Schulausflug zur FAU in Nürnberg. ...,1,0,0,0,0,0
2,Ich bin hier einfach reingelaufen. An einem Ab...,0,1,1,1,0,1
3,"Email von einer Kollegin. Gestern, gemeinsam m...",0,1,1,0,0,1
4,Über Arbeitskollegen. Mir hat letztens jemand ...,1,0,0,1,1,0


#### Openness: Roberta Base

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)

# Create a ClassificationModel
model_bin = ClassificationModel(
    "roberta", "roberta-base", args=model_args_bin, use_cuda = True
)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.6771695818219866)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.7644707871564383, 'tp': 5, 'tn': 9, 'fp': 0, 'fn': 2, 'auroc': 1.0, 'auprc': 1.0, 'f1': 0.8333333333333333, 'eval_loss': 0.630767822265625}


#### Openness: Bert German Cased

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'bert', 'bert-base-german-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.18259361386299133)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.629940788348712, 'tp': 6, 'tn': 7, 'fp': 2, 'fn': 1, 'auroc': 0.9365079365079365, 'auprc': 0.9367346938775509, 'f1': 0.7999999999999999, 'eval_loss': 0.4892001152038574}


#### Openness distilbert

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'distilbert', 'distilbert-base-multilingual-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'classifie

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.40537591661725725)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.363696483726654, 'tp': 3, 'tn': 8, 'fp': 1, 'fn': 4, 'auroc': 0.9047619047619048, 'auprc': 0.859750566893424, 'f1': 0.5454545454545454, 'eval_loss': 0.625035285949707}


## Agreeableness

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Section
from sklearn.metrics import f1_score
import pandas as pd
import torch
print(torch.cuda.is_available())
train_df = pd.read_csv('Joel\\datasets\\sds\\transf\\train.csv')
eval_df = pd.read_csv('Joel\\datasets\\sds\\transf\\test.csv')
train_df['labels'] = train_df.ver_bin
eval_df['labels'] = eval_df.ver_bin
train_df.head()

True


,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch eine Freundin von mir. Ich habe sehr gel...,1,1,1,1,0,1
1,Durch einen Schulausflug zur FAU in Nürnberg. ...,1,0,0,0,0,0
2,Ich bin hier einfach reingelaufen. An einem Ab...,0,1,1,1,0,1
3,"Email von einer Kollegin. Gestern, gemeinsam m...",0,1,1,0,0,0
4,Über Arbeitskollegen. Mir hat letztens jemand ...,1,0,0,1,1,1


### Agreeableness: roberta base

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)

# Create a ClassificationModel
model_bin = ClassificationModel(
    "roberta", "roberta-base", args=model_args_bin, use_cuda = True
)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.45843328748430523)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.6201736729460423, 'tp': 10, 'tn': 3, 'fp': 3, 'fn': 0, 'auroc': 0.75, 'auprc': 0.8436979686979688, 'f1': 0.8695652173913044, 'eval_loss': 0.7267208099365234}


### Agreeableness: bert german cased

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'bert', 'bert-base-german-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.2460439988545009)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.6201736729460423, 'tp': 10, 'tn': 3, 'fp': 3, 'fn': 0, 'auroc': 0.9166666666666667, 'auprc': 0.954040404040404, 'f1': 0.8695652173913044, 'eval_loss': 0.6556849479675293}


### Agreeableness distilbert

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'distilbert', 'distilbert-base-multilingual-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classif

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.3763035774230957)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.7333333333333333, 'tp': 9, 'tn': 5, 'fp': 1, 'fn': 1, 'auroc': 0.8500000000000001, 'auprc': 0.9187698412698413, 'f1': 0.9, 'eval_loss': 0.5418386459350586}


## Conscientousness

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Section
from sklearn.metrics import f1_score
import pandas as pd
import torch
print(torch.cuda.is_available())
train_df = pd.read_csv('Joel\\datasets\\sds\\transf\\train.csv')
eval_df = pd.read_csv('Joel\\datasets\\sds\\transf\\test.csv')
train_df['labels'] = train_df.gew_bin
eval_df['labels'] = eval_df.gew_bin
train_df.head()

True


,text,neuro_bin,extra_bin,off_bin,ver_bin,gew_bin,labels
0,Durch eine Freundin von mir. Ich habe sehr gel...,1,1,1,1,0,0
1,Durch einen Schulausflug zur FAU in Nürnberg. ...,1,0,0,0,0,0
2,Ich bin hier einfach reingelaufen. An einem Ab...,0,1,1,1,0,0
3,"Email von einer Kollegin. Gestern, gemeinsam m...",0,1,1,0,0,0
4,Über Arbeitskollegen. Mir hat letztens jemand ...,1,0,0,1,1,1


### Conscientousness: roberta base

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)

# Create a ClassificationModel
model_bin = ClassificationModel(
    "roberta", "roberta-base", args=model_args_bin, use_cuda = True
)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.6218726021902902)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.5945883900105632, 'tp': 5, 'tn': 7, 'fp': 0, 'fn': 4, 'auroc': 0.7857142857142857, 'auprc': 0.8781144781144783, 'f1': 0.7142857142857143, 'eval_loss': 0.6503791809082031}


### Conscientousness german bert cased 

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'bert', 'bert-base-german-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.18287421805518014)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.49206349206349204, 'tp': 7, 'tn': 5, 'fp': 2, 'fn': 2, 'auroc': 0.873015873015873, 'auprc': 0.9216200466200466, 'f1': 0.7777777777777778, 'eval_loss': 1.313831388950348}


### Conscientousness distilbert

In [ ]:
# Optional model configuration
model_args_bin = ClassificationArgs(num_train_epochs=10)
# Create a ClassificationModel
model_bin = ClassificationModel(
    'distilbert', 'distilbert-base-multilingual-cased', args=model_args_bin, use_cuda = True)
# Train the model
model_bin.train_model(train_df)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classif

  0%|          | 0/56 [00:00<?, ?it/s]

C:\Users\Max\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

(70, 0.43699798583984373)

In [ ]:
result, model_outputs, wrong_predictions = model_bin.eval_model(eval_df, f1=f1_score)
print(result)

  0%|          | 0/16 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

{'mcc': 0.49206349206349204, 'tp': 7, 'tn': 5, 'fp': 2, 'fn': 2, 'auroc': 0.8888888888888888, 'auprc': 0.9253086419753086, 'f1': 0.7777777777777778, 'eval_loss': 0.5156059265136719}
